# Query Environments
This notebook demonstrates the use of the list and get functions on Managed KX Resources.


In [ ]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env import *

In [ ]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

# Environment

In [ ]:
resp = get_kx_environment(client)
resp.pop('ResponseMetadata', None)

infra = resp.get('dedicatedServiceAccountId', "")
print(f"ID: {resp['environmentId']} ({resp['status']}): {resp['name']} ({infra})")

# Databases and Changesets

In [ ]:
try:
    dbs = list_kx_databases(client, environmentId=ENV_ID)
except:
    dbs=[]

print(f"Environment: {ENV_ID}: Databases found: {len(dbs)}")
print()

if len(dbs) > 0:
    # Sort databases by name
    dbs = sorted(dbs, key=lambda d: d['databaseName']) 

    for d in dbs:
        db_name = d['databaseName']
        c_set_list = []
        note_str = ""

        db=client.get_kx_database(environmentId=ENV_ID, databaseName=db_name)
        db.pop('ResponseMetadata', None)

        try:
            resp = client.list_kx_changesets(environmentId=ENV_ID, databaseName=db_name)
            c_set_list = resp['kxChangesets']
        except:
            note_str = "<<Could not get changesets>>"
            break

        print("="*100)        
        print(f"Database: {db_name}, Changesets: {len(c_set_list)} {note_str}")
        print(f"{db.get('description', '')}")
        print(f"Bytes: {db['numBytes']:,} Changesets: {db['numChangesets']:,} Files: {db['numFiles']:,}")
        print("-"*100)        

        c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

        for c in c_set_list:
            c_set_id = c['changesetId']
            print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
            c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=db_name, changesetId=c_set_id)['changeRequests']

            chs_pdf = pd.DataFrame.from_dict(c_rqs)# .style.hide(axis='index')
            display(chs_pdf)
        print("")
                    
    print("="*100)

# Clusters

## Type: HDB

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="HDB")
display(cdf)

## Type: RDB

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="RDB")
display(cdf)

## Type: GATEWAY

In [ ]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="GATEWAY")
display(cdf)

In [ ]:
print( f"Last Run: {datetime.datetime.now()}" )